In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import torch
from models.vcn_32 import VCN32

In [18]:
vcn32 = VCN32()

In [19]:
vcn32

VCN32(
  (down1): DownSample(
    (conv_pool): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (down2): DownSample(
    (conv_pool): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (down3): DownSample(
    (conv_pool): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (down4): DownSample(
    (conv_pool): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2

In [21]:
inp = torch.rand((1, 3, 224, 224))
inp = torch.rand((1, 3, 223, 223))
out = vcn32(inp)
out.shape

torch.Size([223, 223])


torch.Size([1, 2, 192, 192])

In [22]:
out

tensor([[[[-0.0087, -0.0087, -0.0085,  ..., -0.0086, -0.0085, -0.0086],
          [-0.0085, -0.0086, -0.0085,  ..., -0.0087, -0.0085, -0.0086],
          [-0.0086, -0.0085, -0.0087,  ..., -0.0085, -0.0087, -0.0087],
          ...,
          [-0.0084, -0.0087, -0.0086,  ..., -0.0084, -0.0087, -0.0086],
          [-0.0084, -0.0085, -0.0087,  ..., -0.0086, -0.0087, -0.0086],
          [-0.0085, -0.0084, -0.0087,  ..., -0.0085, -0.0086, -0.0087]],

         [[-0.0086, -0.0085, -0.0087,  ..., -0.0085, -0.0087, -0.0086],
          [-0.0085, -0.0085, -0.0086,  ..., -0.0085, -0.0085, -0.0085],
          [-0.0087, -0.0084, -0.0087,  ..., -0.0086, -0.0088, -0.0087],
          ...,
          [-0.0086, -0.0084, -0.0085,  ..., -0.0087, -0.0087, -0.0085],
          [-0.0087, -0.0087, -0.0086,  ..., -0.0086, -0.0085, -0.0085],
          [-0.0087, -0.0085, -0.0085,  ..., -0.0086, -0.0085, -0.0087]]]],
       grad_fn=<ConvolutionBackward0>)